In [46]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import numpy as np
import matplotlib.pyplot as plt
import precision_recall_cutoff 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-wagner'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
file_key = "turnover.csv"

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the csv file
turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [1]:
#Changing to dummy variables
turnover = pd.concat([turnover.drop(columns = ['sales'], axis = 1), pd.get_dummies(turnover['sales'])], axis = 1)
turnover = pd.concat([turnover.drop(columns = ['salary']), pd.get_dummies(turnover['salary'])], axis = 1)
turnover.head()

NameError: name 'pd' is not defined

In [40]:
#creating interactions
turnover['Interaction_1'] = np.where((turnover['satisfaction_level'] <= 0.465) & (turnover['number_project'] <= 2.5) & (turnover['last_evaluation'] <= 0.575), 1, 0)
turnover['Interaction_2'] = np.where((turnover['satisfaction_level'] <= 0.465) & (turnover['number_project'] >= 2.5) & (turnover['satisfaction_level'] <= 0.115), 1, 0)
turnover['Interaction_3'] = np.where((turnover['satisfaction_level'] >= 0.465) & (turnover['time_spend_company'] <= 4.5) & (turnover['average_montly_hours'] <= 290.5), 1, 0)

In [41]:
#Defining the input and target variables
X = turnover.drop(columns = ['left', 'salary'], axis = 1)
Y = turnover['left']

## Spliting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [42]:
X_train = X_train[['Interaction_3', 'Interaction_1', 'satisfaction_level', 'time_spend_company', 'number_project']]
X_test = X_test[['Interaction_3', 'Interaction_1', 'satisfaction_level', 'time_spend_company', 'number_project']]

Ada_param_grid = {'n_estimators': [100, 300, 500],
'base_estimator__min_samples_split': [10, 15],
'base_estimator__min_samples_leaf': [5, 7],
'base_estimator__max_depth': [3, 5, 7],
'learning_rate': [0.001, 0.01, 0.1]}

#running grid search with 3 folds
Ada_grid_search = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), Ada_param_grid, cv = 3, scoring = 'f1', n_jobs = -1).fit(X_train, Y_train)

Ada_grid_search.best_params_

{'base_estimator__max_depth': 5,
 'base_estimator__min_samples_leaf': 5,
 'base_estimator__min_samples_split': 15,
 'learning_rate': 0.01,
 'n_estimators': 500}

In [47]:
#Building model
Ada_model = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(min_samples_split = 15, min_samples_leaf = 5, max_depth = 5), n_estimators = 500 , learning_rate = 0.01).fit(X_train, Y_train)

#test
Ada_pred = Ada_model.predict_proba(X_test)[:, 1]

#labels
Ada_labels = precision_recall_cutoff.precision_recall_cutoff(Y_test, Ada_pred)

#classification report
print(classification_report(Y_test, Ada_labels))


              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2286
           1       0.95      0.95      0.95       714

    accuracy                           0.97      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.97      0.97      0.97      3000



In [48]:
#using 3 folds running RandomizedSearchCV 
Ada_random_search = RandomizedSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), Ada_param_grid, cv = 3, scoring = 'f1', n_jobs = -1,  n_iter = 50).fit(X_train, Y_train)

## Extracting the best hyper-parameters
Ada_random_search.best_params_

{'n_estimators': 300,
 'learning_rate': 0.01,
 'base_estimator__min_samples_split': 15,
 'base_estimator__min_samples_leaf': 5,
 'base_estimator__max_depth': 7}

In [49]:
#Building model
Ada_model = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(min_samples_split = 15, min_samples_leaf = 5, max_depth = 7), n_estimators = 300 , learning_rate = 0.01).fit(X_train, Y_train)

#test
Ada_pred = Ada_model.predict_proba(X_test)[:, 1]

#labels
Ada_labels = precision_recall_cutoff.precision_recall_cutoff(Y_test, Ada_pred)

#classification report
print(classification_report(Y_test, Ada_labels))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2286
           1       0.95      0.95      0.95       714

    accuracy                           0.98      3000
   macro avg       0.97      0.97      0.97      3000
weighted avg       0.98      0.98      0.98      3000



In [ ]:
#we should use the random search because it has the higher overall accuracy